### 四. @Aspect 使用

@Aspectj 想普通的 java 注解一样, 提供声明式面向切面编程. spring AOP 使用和 Aspecj 相同的注解进行事 AOP. 即使用 Aspectj 的切点解析包和匹配包, 但 aop runtime 是 spring 自己的

1. 开启 @Aspectj 自动代理    
   spring 如果发现一个 bean 需要被代理, 则自动创建一个代理对象对 bean 增强  
	* 使用 `@EnableAspectJAutoProxy` 注解开启自动代理发现   
	   因为 @Configuration 注解的类是 beans.xml 的代替, 所以 `@EnableAspectJAutoProxy` 要加在 `@Configuration` 上
	```java
	@Configuration
	@EnableAspectJAutoProxy
	public class AppConfig {

	}
	```

2. 声明一个切面   
  被 @AspectJ 注解的类, 会被 spring 发现并自动创建代理. 
	```java
	package org.xyz;
	import org.aspectj.lang.annotation.Aspect;

	@Aspect
	public class NotVeryUsefulAspect {

	}
	```

3. 定义切点   
	1. 声明切点
      切点注释在方法上, 方法名是"切点签名" (方法返回值应该为 void), `@Pointcut` 注解里是切点表达式, 表示对哪些方法进行增强
	```java
	@Pointcut("execution(* transfer(..))") // the pointcut expression
	private void anyOldTransfer() {} // the pointcut signature
	```
     因为 spring aop 使用代理实现切面, 所以对于 JDK 动态代理, 只能代理实现 public interface 的方法进行切面增强; 对于 CGLib 代理, 可以代理 package visible 的方法 (public 或 protected) 

    2. 切点表达式 
    	* execution : 匹配 join point 的执行, 例如 "execution(* hello(..))" 表示匹配所有目标类中的 hello() 方法. 这个是最基本的 pointcut 标志符.
    	   ```java
			// 匹配指定包中的所有的方法
			execution(* com.xys.service.*(..))

			// 匹配当前包中的指定类的所有方法
			execution(* UserService.*(..))

			// 匹配指定包中的所有 public 方法
			execution(public * com.xys.service.*(..))

			// 匹配指定包中的所有 public 方法, 并且返回值是 int 类型的方法
			execution(public int com.xys.service.*(..))

			// 匹配指定包中的所有 public 方法, 并且第一个参数是 String, 返回值是 int 类型的方法
			execution(public int com.xys.service.*(String name, ..))
    	   ```  
    	* `within` :   
    	   匹配特定包下的所有 join point, 例如 within(com.xys.*) 表示 com.xys 包中的所有连接点, 即包中的所有类的所有方法. 而 within(com.xys.service.*Service) 表示在 com.xys.service 包中所有以 Service 结尾的类的所有的连接点.
    	   ```java
			// 匹配指定包中的所有的方法, 但不包括子包
			within(com.xys.service.*)

			// 匹配指定包中的所有的方法, 包括子包
			within(com.xys.service..*)

			// 匹配当前包中的指定类中的方法
			within(UserService)

			// 匹配一个接口的所有实现类中的实现的方法
			within(UserDao+)
    	   ```
    	* `this 与 target`  
           this 的作用是匹配一个 bean, 这个 bean(Spring AOP proxy) 是一个给定类型的实例(instance of). 而 target 匹配的是一个目标对象(target object, 即需要织入 advice 的原始的类), 此对象是一个给定类型的实例(instance of).
        * `bean`    
           匹配 bean 名字为指定值的 bean 下的所有方法 例如
           ```java
			bean(*Service) // 匹配名字后缀为 Service 的 bean 下的所有方法
			bean(myService) // 匹配名字为 myService 的 bean 下的所有方法
           ```
        * `args`  
           匹配参数满足要求的的方法
        * `@annotation`  
		   匹配由指定注解所标注的方法, 例如:
		   ```java
			@Pointcut("@annotation(com.xys.demo1.AuthChecker)")
			public void pointcut() {
			}
		   ```

4. Advice 声明  
  Advice 和 切点表达式组合, 运行在 poiuntcut 之前(before), 之后(after), 或是包围(around);    
  切点表达式可以是: 
	* "一个切点的命名引用"
	* 也可以写完整的"切点表达式" 
	1. Before Advice  
      在 @Aspect 声明的切面中, 使用 @Before 声明 before 增强    
    ```java
	@Aspect
	public class BeforeExample {

		// 直接引用切点的方法名
	    @Before("com.xyz.myapp.CommonPointcuts.dataAccessOperation()")
	    public void doAccessCheck() {
	        // ...
	    }

	    // 切点表达式
	    @Before("execution(* com.xyz.myapp.dao.*.*(..))")
	    public void doAccessCheck() {
	        // ...
	    }
	}
    ```
    2. After Advice (如果被增强方法抛出异常, 这种 advice 就不会执行)  
      用 `@AfterReturning` 声明后置增强. 有时想使用被增强方法的返回值, 可以用 returning 属性指定返回值的名称, 该名称和 advice 的方法参数名相同
    ```java
	@Aspect
	public class AfterReturningExample {

	    @AfterReturning(
	        pointcut="com.xyz.myapp.CommonPointcuts.dataAccessOperation()",
	        returning="retVal")
	    public void doAccessCheck(Object retVal) {
	        // ...
	    }
	}
    ```
    3. After Throwing Advice  
      有时想在被增强方法抛出异常后执行增强, 可以用 `@AfterThrowing` 声明. 如果想判断抛出异常为某特定类型异常时, 增强方法中加入特定的异常类型参数即可  
	```java
	// 对所有异常进行 AfterThrowing 增强
	@Aspect
	public class AfterThrowingExample {

	    @AfterThrowing("com.xyz.myapp.CommonPointcuts.dataAccessOperation()")
	    public void doRecoveryActions() {
	        // ...
	    }
	}

	// 对特定类型的异常进行 AfterThrowing 增强  
	@Aspect
	public class AfterThrowingExample {

	    @AfterThrowing(
	        pointcut="com.xyz.myapp.CommonPointcuts.dataAccessOperation()",
	        throwing="ex")
	    public void doRecoveryActions(DataAccessException ex) {
	        // ...
	    }
	}
    ```
    4. After (Finally) Advice (相当于吧增强方法写在 finnaly 代码块中)
    需要处理正常返回和异常返回. 一般用于释放资源等目的
    ```java
	@Aspect
	public class AfterFinallyExample {

	    @After("com.xyz.myapp.CommonPointcuts.dataAccessOperation()")
	    public void doReleaseLock() {
	        // ...
	    }
	}
    ```
